# Mapa de la quinta entrega

In [ ]:
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate
from numpy import mean
import geopandas as gpd
import pandas as pd
import folium
from folium import Choropleth, Marker
from IPython.display import display
from folium.plugins import HeatMap

warnings.filterwarnings('ignore')

In [ ]:
t32023 = pd.read_csv("precios_t32023.csv")
t42023 = pd.read_csv("precios_t42023.csv")
t12023 = pd.read_csv("precios_t12023.csv")
t22023 = pd.read_csv("precios_t22023.csv")

d2023 = pd.concat([t32023, t42023, t12023, t22023], ignore_index=True)
d2023.head()

mapa_col = gpd.read_file("COLOMBIA.shp")
mapa_col.head()

mapa_col['DPTO_CNMBR'] = mapa_col['DPTO_CNMBR'].str.replace('?', 'Ñ')
mapa_col['DPTO_CNMBR'] = mapa_col['DPTO_CNMBR'].str.replace('ARCHIPIELAGO DE SAN ANDRES, SANTA CATALINA Y PROVIDENCIA', 'ARCHIPIELAGO DE SAN ANDRES')

mapa_col = mapa_col.rename(columns={'DPTO_CNMBR': 'DEPARTAMENTO'})
precios_promedio = d2023.groupby('DEPARTAMENTO')['VALOR PRECIO'].mean().reset_index()
precios_promedio = precios_promedio.rename(columns={'VALOR PRECIO': 'PROMEDIO_PRECIO'})
datos_unidos = mapa_col.merge(precios_promedio, on='DEPARTAMENTO', how='left')

In [ ]:
m = folium.Map(location=[4.5709, -74.2973], zoom_start=7, tiles='CartoDB positron')

folium.Choropleth(
    geo_data=datos_unidos,
    name='choropleth',
    data=datos_unidos,
    columns=['DEPARTAMENTO', 'PROMEDIO_PRECIO'],
    key_on='feature.properties.DEPARTAMENTO',
    fill_color='Spectral',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Precio Promedio',
    highlight=True,
).add_to(m)

for _, row in datos_unidos.iterrows():
    folium.Marker(
        location=[row.geometry.centroid.y, row.geometry.centroid.x],
        icon=None,
        popup=f"{row['DEPARTAMENTO']}: {row['PROMEDIO_PRECIO']:.2f}",
    ).add_to(m)

folium.LayerControl().add_to(m)

m
